In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.gridspec as gridspec
import numpy as np
from pathlib import Path

In [2]:
# Current working directory (notebook folder)
notebook_path = Path.cwd()  # MLT/code/data_processing/
base_path = notebook_path.parent.parent  # Go up to MLT/

functions_path = notebook_path.parent / "000_Functions.ipynb"
bronze_path = base_path / "data" / "bronze"
plots_path = base_path / "plots"

In [3]:
# Load CSVs
df = pd.read_csv( bronze_path / "bronze_saber_4d_tempoaral_agg.csv")

In [4]:
df['year_month'] = pd.to_datetime(df['year_month']).dt.to_period('M')

In [5]:
df = df[['year_month', 'altitude_bin', 'latitude_bin', 'longitude_bin', 'ktemp']]

In [6]:
# Pivot the dataframe: rows = tpaltitude, columns = tplatitude
pivot_lat = df.pivot_table(index='altitude_bin', columns='latitude_bin', values='ktemp')

# Create X, Y meshgrid from the pivot index and columns
X_lat, Y_lat = np.meshgrid(pivot_lat.columns.values, pivot_lat.index.values)

# Extract Z values (ktemp)
Z_lat = pivot_lat.values

In [7]:
# Pivot the dataframe: rows = tpaltitude, columns = tplatitude
pivot_lon = df.pivot_table(index='altitude_bin', columns='longitude_bin', values='ktemp')

# Create X, Y meshgrid from the pivot index and columns
X_lon, Y_lon = np.meshgrid(pivot_lon.columns.values, pivot_lon.index.values)

# Extract Z values (ktemp)
Z_lon = pivot_lon.values

In [8]:
norm = Normalize(vmin=165, vmax=300)  # Set to match desired common scale
levels = np.linspace(165, 300, 60)    # Shared contourf levels
line_levels = np.linspace(165, 300, 15)  # Shared contour levels

# Set up figure and gridspec (note: no sharey here)
fig = plt.figure(figsize=(15, 4), dpi=1000)
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

# Create subplots with shared y-axis
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharey=ax1)
cax = fig.add_subplot(gs[2])  # Colorbar axis

# First subplot: Latitude vs Altitude
cf_lat = ax1.contourf(X_lat, Y_lat, Z_lat, cmap='RdBu_r', norm=norm, levels=levels)
contours_lat = ax1.contour(X_lat, Y_lat, Z_lat, levels=line_levels, colors='black', linewidths=0.5)
ax1.clabel(contours_lat, inline=True, fontsize=5)
ax1.set_ylabel('Altitude (km)')
ax1.set_xlabel('Latitude (°)')
ax1.text(-0.1, 1.05, "(a)", transform=ax1.transAxes, fontsize=8, fontweight='bold')
# ax1.set_title('Latitude Dimension')

# Second subplot: Longitude vs Altitude
cf_lon = ax2.contourf(X_lon, Y_lon, Z_lon, cmap='RdBu_r', norm=norm, levels=levels)
contours_lon = ax2.contour(X_lon, Y_lon, Z_lon, levels=line_levels, colors='black', linewidths=0.5)
ax2.clabel(contours_lon, inline=True, fontsize=5)
ax2.set_xlabel('Longitude (°)')
ax2.text(-0.1, 1.05, "(b)", transform=ax2.transAxes, fontsize=8, fontweight='bold')
# ax2.set_title('Longitude Dimension')

# Shared colorbar on the right
cbar = fig.colorbar(cf_lon, cax=cax)
cbar.set_label('Temperature (K)')

plt.tight_layout()
plt.savefig(plots_path / "Altitude_vs_Coordinates.png", format='png', dpi=300, bbox_inches='tight')
plt.show()

C:\Users\mvana\AppData\Local\Temp\ipykernel_2156\4129076679.py:35: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
